# OpenFisca-UK

This book contains an introduction to using OpenFisca-UK to model UK taxes and benefits. It is currently a work in progress and may be added to. OpenFisca-UK is a microsimulation model of the UK tax and benefit system: it is a model which calculates variable values over UK entities from given policy parameters and structures. In practice, this gives it two main uses: calculating statistics under current tax and benefit law, and simulating effects of potential new changes to the legislation.

## Short demo

### Baseline estimation

Calculating, for example, the total tax (Income Tax and National Insurance) liability by region can be done with the following code:

In [1]:
from openfisca_uk import Microsimulation
import pandas as pd

sim = Microsimulation(mode="frs", year=2018)

df = sim.df(["national_insurance", "income_tax", "people"])

summary = pd.DataFrame(df.groupby(sim.calc("region", map_to="person")).sum()).sort_values(by="people", ascending=False)
summary.national_insurance = summary.national_insurance.apply(lambda x: round(x / 1e+9, 1))
summary.income_tax = summary.income_tax.apply(lambda x: round(x / 1e+9, 1))
summary.people = summary.people.apply(lambda x: round(x / 1e+6, 1))
summary.columns = ["National Insurance (£bn)", "Income Tax (£bn)", "Population (millions)"]
summary

,National Insurance (£bn),Income Tax (£bn),Population (millions)
SOUTH_EAST,9.3,28.3,9.0
LONDON,11.2,35.0,8.9
NORTH_WEST,6.1,13.5,7.2
EAST_OF_ENGLAND,6.2,17.3,6.1
WEST_MIDLANDS,4.6,10.7,5.7
SOUTH_WEST,4.6,12.6,5.5
YORKSHIRE,4.2,10.0,5.4
WALES,4.9,12.3,5.4
EAST_MIDLANDS,4.1,9.8,4.7
SCOTLAND,2.4,5.2,3.1


### Reform evaluation

Below is an example of simulating the effects of a reform (namely, increasing the basic rate of income tax from 20% to 23%). This uses the SPI mode of input - using administrative tax data instead of the FRS household survey.

In [2]:
from openfisca_uk.api import *

def change_tax_parameters(parameters):
    parameters.tax.income_tax.rates.uk.brackets[0].rate.update(
        period=periods.period("year:2019:1"), value=0.23
    )
    return parameters

class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)

baseline = Microsimulation(mode="spi")
reformed = Microsimulation(reform, mode="spi")
revenue = reformed.calc("tax").sum() - baseline.calc("tax").sum()
f"Revenue: £{round(revenue / 1e+9, 1)}bn"

'Revenue: £11.8bn'


```{toctree}
:hidden:
:titlesonly:


Reforms
Weighting
Individuals
```
